# Initial calculation code

In [173]:
import numpy as np

def geom_avg(vals):
    """
    Compute the geometric average of a list of values.
    
    The values need not be a list, but simply anything with a len() and []
    """
    rval=1.0
    count = 0
    for val in vals:
        val = vals[count]
        if val != 0:
            rval *= val
            count+=1
    if count != 0:
        rval = pow(rval, 1.0/count)
    return(rval)

def geom_avg_mat(mat, coeffs = None):
    '''
    Computes the geometric average of the columns of a matrix.  Returns
    an np.array of dimension [nRowsOfMat], i.e. a vector.  
    
    :param mat: Must be an numpy.array of shape [nRows, nCols]
    :param coeffs:  If not None, it is a list like object with nColsOfMat elements.
    We multiply column 0 of mat by coeffs[0], column 1 of mat by coeffs[1], etc
    and then do the geometric average of the columns.  Essentially this weights the
    columns.
    '''
    """
    """
    size = mat.shape[0]
    rval = np.ones([size])
    for row in range(size):
        if np.any(coeffs):
            theRow = mat[row,:] * np.array(coeffs)
        else:
            theRow = mat[row,:]
        rval[row] = geom_avg(theRow)
    return(rval)

def bpriorities(mat, error = 1e-10):
    """
    Calculates priorities using Bill's method
    """
    size = mat.shape[0]
    vec = np.ones([size])
    diff = 1
    count=0
    while diff >= error and count < 100:
        nextv = geom_avg_mat(mat, vec)
        #nextv = nextv/max(nextv)
        diff = max(abs(nextv - vec))
        vec = nextv
        count+=1
    return(vec/sum(vec))

def gm_priorities(mat):
    '''
    Calculates the priorities using the geometric mean method
    :param mat: An numpy.array of dimension [size,size]
    '''
    rval = geom_avg_mat(mat)
    rval = rval / sum(rval)
    return(rval)

def harker_fix(mat):
    """
    Performs Harkers fix on the numpy matrix mat.  It returns a copy with the fix.
    The function does not change the matrix mat.
    :param mat:
    :return:
    """
    nrows = mat.shape[0]
    ncols = mat.shape[1]
    rval = mat.copy()
    for row in range(nrows):
        val = 1
        for col in range(ncols):
            if col != row and mat[row,col]==0:
                val+=1
        rval[row,row]=val
    return(rval)

def largest_eigen(mat, error = 1e-10, use_harker = False):
    if use_harker:
        mat = harker_fix(mat)
    size = mat.shape[0]
    vec = np.ones([size])
    diff = 1
    while diff > error:
        nextv = np.matmul(mat, vec)
        nextv = nextv/sum(nextv)
        diff = max(abs(nextv - vec))
        vec = nextv
    return(vec)

def priority_error(pwmat, privec):
    rval = 0
    diffsum = 0
    count = 0
    size = pwmat.shape[0]
    for i in range(0, size):
        for j in range (0, size):
            if pwmat[i,j] != 0:
                diffsum += (pwmat[i,j] - privec[i]/privec[j])**2
                count += 1
    if count == 0:
        return 0
    else:
        return diffsum ** (1.0/count)
    
def ratio_mat(pv):
    size = len(pv)
    rval = np.identity(n=size)
    for row in range(size):
        for col in range(size):
            rval[row,col]=pv[row]/pv[col]
    return rval

# First round of calculations

## The matrix and largest eigen

In [174]:
mat = np.array([[1, 2, 1/9], [1/2, 1, 2], [9, 1/2, 1]])
mat

array([[ 1.        ,  2.        ,  0.11111111],
       [ 0.5       ,  1.        ,  2.        ],
       [ 9.        ,  0.5       ,  1.        ]])

In [176]:
leigen = largest_eigen(mat, error=1e-15)
leigen

array([ 0.18598961,  0.30706208,  0.50694832])

## Calculate the mismatch of the priority vector vs the pairwise matrix

In [177]:
priority_error(mat, leigen)

1.5300986463689501

## Check gradient of error function
It should be all zeroes at `leigen` if that is a minimimum.

In [181]:
my_error = lambda x: priority_error(mat, x)

In [182]:
from scipy.misc import derivative
from scipy.optimize import approx_fprime, minimize

In [183]:
eps = 1e-14
approx_fprime(leigen, my_error, eps)

array([ 0.57731597,  0.        , -0.19984014])

## Minimize priority_error(mat, *)
**It is not all zeroes, it looks like we can get a smaller error!**

In [102]:
rval = minimize(my_error, leigen, bounds=[(.01, 1), (.01, 1), (.01, 1)])

In [106]:
newpv = rval.x / sum(rval.x)
newpv

array([ 0.08367381,  0.23820374,  0.67812245])

In [186]:
# This is not the same, let's see them both together
leigen, newpv

(array([ 0.18598961,  0.30706208,  0.50694832]),
 array([ 0.08367381,  0.23820374,  0.67812245]))

In [187]:
# Let's look at the error of the eigen vs newpv
my_error(leigen), my_error(newpv)

(1.5300986463689501, 1.3722470115776764)

In [189]:
# Well that is surprising, approximately a 10% decline in error
# Let's consider the ratio matrix for both eigen and newpv
ratio_mat(leigen), None, ratio_mat(newpv)

(array([[ 1.        ,  0.60570686,  0.36688081],
        [ 1.65096362,  1.        ,  0.60570686],
        [ 2.72568089,  1.65096362,  1.        ]]),
 None,
 array([[ 1.        ,  0.35126994,  0.12339042],
        [ 2.84681345,  1.        ,  0.3512695 ],
        [ 8.104357  ,  2.84681703,  1.        ]]))

In [113]:
ratio_mat(pv)

array([[ 1.        ,  0.60570686,  0.36688081],
       [ 1.65096362,  1.        ,  0.60570686],
       [ 2.72568089,  1.65096362,  1.        ]])

In [114]:
my_error(rval.x)

1.3722470115776764

## Try doing a similar calculation with my new priority calculation

In [120]:
bpv=bpriorities(mat)

In [122]:
pv, bpv, newpv

(array([ 0.18598961,  0.30706208,  0.50694832]),
 array([ 0.18598961,  0.30706208,  0.50694832]),
 array([ 0.08367381,  0.23820374,  0.67812245]))

Whoops, I forgot my new calculation agrees with the standard eigen for 3x3 matrices, let's just check for the transpose as well, for giggles

In [133]:
tbpv = bpriorities(mat.transpose())
ibpv = 1/bpv
ibpv = ibpv / sum(ibpv)
tbpv, ibpv

(array([ 0.50694832,  0.30706208,  0.18598961]),
 array([ 0.50694832,  0.30706208,  0.18598961]))

In [134]:
largest_eigen(mat.transpose())

array([ 0.50694832,  0.30706208,  0.18598961])

In [160]:
def bill_iter(mat, p):
    rval = p/sum(p)
    size = len(p)
    for i in range(size):
        c = 1
        for j in range(size):
            if mat[i,j] >= 1:
                err = mat[i,j]/(p[i]/p[j])
            else:
                err = mat[j,i]/(p[j]/p[i])
            c *= err
        c = c ** (1/(2*(size-1)))
        rval[i]*=c
    rval=rval/sum(rval)
    return rval

In [165]:
p1=bill_iter(mat, newpv)
p2=bill_iter(mat, p1)
p3=bill_iter(mat, p2)
p4=bill_iter(mat, p3)

In [166]:
newpv, p1, p2, p3, p4

(array([ 0.08367381,  0.23820374,  0.67812245]),
 array([ 0.07469307,  0.3199674 ,  0.60533952]),
 array([ 0.0744782 ,  0.43346543,  0.49205637]),
 array([ 0.08331022,  0.55091165,  0.36577813]),
 array([ 0.10633691,  0.63085331,  0.26280978]))

In [170]:
my_error(p1)

1.4122226603944246